In [98]:
#traces
import pandas as pd
import numpy as np
import re
import sys
import math
#for ordered dict for traces implementation
from collections import OrderedDict

from datetime import datetime
from datetime import timedelta
#import sources.endomondolib as endo
#import sources.pysparkconvenience as ps
from numpy import array
from math import sqrt
from pyspark.sql.types import DoubleType
#for timedelta manipulation
from math import fabs

#kmeans
from pyspark.mllib.clustering import KMeans, KMeansModel
#from pyspark.mllib.linalg import Vectors
#from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler

from  pyspark.ml.feature import MinMaxScaler
from pyspark.ml.linalg import Vectors, VectorUDT


from pyspark.sql.functions import *
#from pyspark.mllib.clustering import KMeans, KMeansModel
from pyspark.ml.clustering import KMeans, KMeansModel
from pyspark.sql import DataFrameReader
from pyspark.sql import DataFrame
from pyspark.sql.functions import count
from pyspark.sql.window import Window
from pyspark.sql import SQLContext
from IPython.display import display, HTML
from pyspark.sql.functions import col
#from pyspark.mllib.linalg import Vectors
from pyspark.sql import HiveContext
from pyspark.ml.feature import VectorAssembler

# Disable warnings, set Matplotlib inline plotting and load Pandas package
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
pd.options.display.mpl_style = 'default'



from pyspark import SparkContext
#sc =SparkContext()


pyspark_submit_args = os.environ.get("PYSPARK_SUBMIT_ARGS", "")
if not "pyspark-shell" in pyspark_submit_args: pyspark_submit_args += " pyspark-shell"
os.environ["PYSPARK_SUBMIT_ARGS"] = pyspark_submit_args

#%load_ext autotime
from pyspark.sql import SQLContext
sqlContext = SQLContext(sc)

#del min
#del max

data_run='/home/ubuntu/run_data_with_hr_spd.csv'
data_users='/Users/momori/dse/maomori/thesis/git/fitness_capstone/data/run_with_hr_users.csv'

sqlContext = HiveContext(sc)

from rdp import rdp

In [99]:
#create pyspark dataframe from csv
def df_from_csv(csv_file):
    text = sc.textFile(csv_file)\
        .map(lambda line: line.split(","))
    #didn't work with take(1). believe returns 
    #different object then first()
    schema = text.first()
    data = text.filter(lambda x: x != schema)
    df = sqlContext.createDataFrame(data, schema)
    return df

def change_column_names(df, old_names, new_names):
    pass
    return df;

##input
##   string: 2013-02-22 21:38:45
##output: 
##   list: [u'2013', u'02', u'22', u'21', u'38', u'45']
def datetime_to_trace_time(timestamp):
    #strip double quotes
    timestamp = timestamp.replace('"', '')
    r_list = re.split('[- :]', timestamp)
    return [int(x) for x in r_list]

In [292]:
df_runs = df_from_csv(data_run)
df_runs.count()

300000

In [101]:
sub_df = df_runs.limit(500)

In [102]:
#minMaxScaler wrapper since originalMin/Max is only implemented in 2.0
class scaler_wrapper():
    mmModel = ''
    originalMin = ''
    originalMax = ''   
    
    def __init__(self, inputCol, outputCol):
        self.mmModel = MinMaxScaler(inputCol=inputCol, outputCol=outputCol)
        self.in_column = inputCol
        
    def get_input_col_name(self):
        return self.mmModel.getInputCol()

    def getMax(self):
        return self.mmModel.getMax()
        
    def getMin(self):
        return self.mmModel.getMin()
    
    def describe(self):
        print 'describe'
    
    def fit(self, df):
        col = self.mmModel.getInputCol()
        print 'in fit'
        print type(df.select(col).rdd)
        self.originalMin = df.select(col).rdd.flatMap(lambda x: x[0]).min()
        self.originalMax = df.select(col).rdd.flatMap(lambda x: x[0]).max()
        return self.mmModel.fit(df)
    
    #denormalize the value
    def denormalize(self, value):
        v = (value-self.getMin())*\
            (self.originalMax - self.originalMin)*\
            (self.getMax()-self.getMin()) + self.originalMin
        if v or v == 0:
            return v
        else:
            return -999
        
    def normalize(self, value):
        pass

In [103]:
#minMaxScaler wrapper since originalMin/Max is only implemented in 2.0
class scaler_wrapper():
    mmModel = ''
    originalMin = ''
    originalMax = ''   
    
    def __init__(self, inputCol, outputCol):
        self.mmModel = MinMaxScaler(inputCol=inputCol, outputCol=outputCol)
        self.in_column = inputCol
        
    def get_input_col_name(self):
        return self.mmModel.getInputCol()

    def getMax(self):
        return self.mmModel.getMax()
        
    def getMin(self):
        return self.mmModel.getMin()
    
    def describe(self):
        print 'describe'
    
    def fit(self, df):
        col = self.mmModel.getInputCol()
        self.originalMin = df.select(col).rdd.flatMap(lambda x: x[0]).min()
        self.originalMax = df.select(col).rdd.flatMap(lambda x: x[0]).max()
        return self.mmModel.fit(df)
    
    #denormalize the value
    def denormalize(self, value):
        v = (value-self.getMin())*\
            (self.originalMax - self.originalMin)*\
            (self.getMax()-self.getMin()) + self.originalMin
        if v or v == 0:
            return v
        else:
            return -999
        
    def normalize(self, value):
        pass

In [104]:
df_runs.schema

StructType(List(StructField("time",StringType,true),StructField("altitude",StringType,true),StructField("heart_rate",StringType,true),StructField("latitude",StringType,true),StructField("longitude",StringType,true),StructField("speed",StringType,true),StructField("workoutid",StringType,true),StructField("id",StringType,true)))

In [105]:
#calculate sse
def compute_sse(a,b):
    total = 0
    if len(a) != len(b):
        print 'bad input'
        return 99999
    else:
        for i in range(len(a)):
            total += math.pow(float(a[i])-float(b[i]),2)
    return math.sqrt(total)



#change column type
def change_type_cols_double(df, list_cols):
    for col_name in list_cols:
        df = df.withColumn(col_name, df[col_name].cast("double"))
    return df.na.fill(0)

#vectorize the column
#keeps the original name
def vectorize_columns(df, list_cols):
    tmp_col_name = 'temp'
    for col_name in list_cols:
        vectorize = udf(lambda vs: Vectors.dense(vs), VectorUDT())
        df = df.withColumn(tmp_col_name, vectorize(df[col_name])).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
    return df

##scaler wrapper usage
#generate normalized dataframe
#keep the original column names
#returns:
#   (df, {col_name:scalerModel, col_name:scalerModel,....})
def normalize_df(df, list_cols):
    df = change_type_cols_double(df, list_cols)
    df = vectorize_columns(df, list_cols)
    tmp_col_name = 'temp'
    r_dict = {}
    index = 0
    for col_name in list_cols:
        #scaler is the wrapper instance
        scaler = scaler_wrapper(inputCol=col_name, outputCol=tmp_col_name)
        scalerModel = scaler.fit(df)
        df = scalerModel.transform(df).drop(col_name)\
            .withColumnRenamed(tmp_col_name, col_name)
        #r_dict[col_name] = scaler
        r_dict[index] = scaler
        index+=1
    return df, r_dict

def denormalize_centers(list_centers, scaler_models):
    new_centers = []
    for center in list_centers:
        n_center = []
        for c in range(len(center)):
            n_center.append(scaler_models[c].denormalize(center[c]))
        new_centers.append(n_center)
    return new_centers



In [301]:
def change_type_cols_double(df, list_cols):
    for col_name in list_cols:
        df = df.withColumn(col_name, df[col_name].cast("double"))
    return df.na.fill(0)

#outputs a list of pyspark dataframes, where each dataframe is grouped by partition_name
def split_by_partition_name(data_points, partition_name='"workoutid"', cols_to_keep = []):
    if not isinstance(data_points, DataFrame):
            print 'please enter spark dataframe object'
            return
    distinct_values = data_points.select(partition_name).rdd.distinct()\
        .flatMap(lambda x:x).collect()
    if cols_to_keep:
        dfs = [data_points.where(data_points[partition_name] == val)\
                   [cols_to_keep]\
                    for val in distinct_values]
    else:
        dfs = [data_points.where(data_points[partition_name] == val)\
                            for val in distinct_values]
    
    return dfs

# print ls_dfs[0].count()
# print ls_dfs[0].collect()[0]
# print len(ls_dfs[0].collect()[0])

#only works for max 3D data. For more, will need to reimplement RDP
def find_pip2(df, epsilon = 0.05):
    list_to_rdp = []
    num_rows = df.count()
    list_rows = df.collect()
    num_cols = len(list_rows[0])
    for row_num in range(num_rows):
        row_list = []
        for col_num in range(num_cols):
            row_list.append(list_rows[row_num][col_num].values.tolist()[0])
        list_to_rdp.append(row_list)
    return rdp(list_to_rdp, epsilon=epsilon)
    
    
def find_pip(list_rows, epsilon=0.5):
    list_to_rdp = []
    num_rows = df.count()
    num_cols = len(list_rows[0])
    for row_num in range(num_rows):
        row_list = []
        for col_num in range(num_cols):
            row_list.append(list_rows[row_num][col_num].values.tolist()[0])
        list_to_rdp.append(row_list)
    return rdp(list_to_rdp, epsilon=epsilon)

In [288]:
#old find_pip
#find_pip2(ls_dfs[0])

[[0.0, 0.0], [0.0, 0.5079365079365079]]

In [114]:
#normalized_df.schema

In [293]:
#Normalize data. retrieve normalized dataframe and the scalerModels used
cols_to_keep = ['"speed"', '"heart_rate"']

normalized_df, scalerModels = normalize_df(df_runs, cols_to_keep)

In [296]:
#split into multiple dfs
ls_dfs = split_by_partition_name(normalized_df, cols_to_keep=cols_to_keep)
print len(ls_dfs)

714


In [ ]:
##original length of the dataframe should be equal to original dataframe
# ---> 300k

In [303]:
#use rdp on the dataframes
ls_collected_dfs = []
count = 0
for df in ls_dfs:
    count += 1
    if count% 3000 == 0:
        print count
    ls_collected_dfs.append(df.collect())
#sc.parallelize(ls_dfs)

KeyboardInterrupt: 

In [299]:
#use rdp on the dataframes
new_points = []
for df in ls_dfs:
    new_points.append(find_pip(df))
print new_points
print len(new_points)

KeyboardInterrupt: 

In [291]:
#new length of data
for k in new_points:
    print len(k),

2 3


# -------test---------